In [25]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go


from scipy.spatial.transform import Rotation as R

from typing import Tuple, List

# After installing ipympl and restarting backend
%matplotlib widget

In [26]:
def calc_pr(rowVec):
    """takes in unit normals and returns pitch and roll euler angles"""
    # project to zy plane
    r, _ = rowVec.shape
    proj_zy = np.hstack((np.zeros((r, 1)), rowVec[:, 1:3]))
    x_sgn = np.sign(rowVec[:, 0])

    # normalize the projected vector
    proj_zy = proj_zy / np.linalg.norm(proj_zy, axis=1)[:, np.newaxis]

    # calculate pitch
    pitch = -np.arctan2(
        proj_zy[:, 1], proj_zy[:, 2]
    )  # per hardware configuration, rotation about x axis has to be flipped

    # calculate roll
    roll = x_sgn * np.arccos(np.diag(proj_zy @ rowVec.T).real)
    for i in range(len(roll)):
        if np.isnan(roll[i]):
            roll[i] = 0.0

    prMat = np.column_stack((pitch, roll))
    return prMat


def rotx(t, deg=None):
    if deg == "deg":
        t = t * np.pi / 180

    ct = np.cos(t)
    st = np.sin(t)

    if not isinstance(t, complex):
        if abs(st) < 1e-15:
            st = 0
        if abs(ct) < 1e-15:
            ct = 0

    R = [[1, 0, 0], [0, ct, -st], [0, st, ct]]
    return R


def roty(t, deg=None):
    if deg == "deg":
        t = t * np.pi / 180

    ct = np.cos(t)
    st = np.sin(t)

    if not isinstance(t, complex):
        if abs(st) < 1e-15:
            st = 0
        if abs(ct) < 1e-15:
            ct = 0

    R = [[ct, 0, st], [0, 1, 0], [-st, 0, ct]]
    return R

In [27]:

def plot_points(points):
    # make marker smaller
    fig = px.scatter_3d(
        points, x=points[:, 0], y=points[:, 1], z=points[:, 2], size_max=0.1
    )

    fig.update_traces(marker_size=1)

    # Set x axis to point down
    fig.update_layout(
        scene=dict(
            xaxis=dict(title="X Axis", showspikes=False),
            yaxis=dict(title="Y Axis", showspikes=False),
            zaxis=dict(title="Z Axis", showspikes=False),
        )
    )

    # Set initial camera view
    fig.update_layout(
        scene_camera=dict(
            up=dict(x=-1, y=0, z=0), center=dict(x=0, y=0, z=0), eye=dict(x=0, y=0, z=3)
        )
    )

    max_range = np.abs(points).max()

    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[-max_range, max_range]),
            yaxis=dict(range=[-max_range, max_range]),
            zaxis=dict(range=[-max_range, max_range]),
        ),
    )

    fig.show()

In [28]:
def calculate_mu_xyz(normal_on_R3, ellipse_params):
    """
    Calculate the probability function: mu_xyz

    Args:
        normal_on_R3: np.array of shape (n_samples, 3) of cartesian points on a sphere generated by picking from a normal distribution
        ellipse_params: array of 3 floats, axes measurements of ellipsoi

    Returns:
        np.array of shape (n_samples, 1), probability function values
    """
    ellipse_a = ellipse_params[0]
    ellipse_b = ellipse_params[1]
    ellipse_c = ellipse_params[2]
    x, y, z = normal_on_R3[:,0], normal_on_R3[:,1], normal_on_R3[:,2]
    mu_xyz = np.sqrt((ellipse_a * ellipse_c * x)**2 + (ellipse_a * ellipse_b * y)**2 + (ellipse_b * ellipse_c * z)**2)
    return mu_xyz
    

def sample_points_ellipsoid(theta_bounds, phi_bounds, n_samples, ellipse_params = [10,9,6.35]):
    """
    Sample points uniformly in the area bounded by the euler angles theta and phi.

    Args:
        theta_bounds: tuple of two floats, bounds of theta in degrees
        phi_bounds: tuple of two floats, bounds of phi in degrees
        n_samples: int, number of samples to generate
        ellipse_params: array of 3 floats, axes measurements of ellipsoid

    Returns:
        final points: np.array of shape (n_samples, 3), points in 3D space on the surface of the ellipsoid.
        final angles: np.array of shape (n_samples, 3), euler angles corresponding to points on the surface of the ellipsoid
    """
    # """Assume bounds are in degrees and return n_samples points within the bounds"""
    theta_bounds = np.deg2rad(theta_bounds)
    phi_bounds = np.deg2rad(phi_bounds)

    ellipse_a = ellipse_params[0]
    ellipse_b = ellipse_params[1]
    ellipse_c = ellipse_params[2]

    points_per_round = n_samples
    final_points = np.empty((0, 3))
    final_angles = np.empty((0, 2))
    
    #calculate angles using unit normals of ellipsoid normals. but final points returned should be normals so we know where the point on the ellipsoid is.
    while True:
        normal_on_R3 = np.random.normal(size=(points_per_round, 3))
        normal_on_R3 = (
            normal_on_R3 / np.linalg.norm(normal_on_R3, axis=1)[:, np.newaxis]
        )

        #transform from point on sphere to point on ellipsoid
        point_on_R3_ellipse = np.array([ellipse_a * normal_on_R3[:,0],ellipse_b * normal_on_R3[:,1],ellipse_c * normal_on_R3[:,2]]).T

        #the max value the probability distribution can have which is the place where the sphere and ellipsoid differ the most: this happens at (0,+-1,0)
        mu_max = ellipse_a*ellipse_b

        #keep point (x',y',z') with probability mu_xyz / mu_max
        random_num = np.random.uniform(size=(points_per_round))
        prob = calculate_mu_xyz(normal_on_R3,ellipse_params) / mu_max 
        mask_prob = random_num < prob

        #remove points based on the above probability
        point_on_R3_ellipse = point_on_R3_ellipse[mask_prob]
        normal_on_R3 = normal_on_R3[mask_prob]

        # Find phi
        z_gt0 = point_on_R3_ellipse[:, 2] > 0  # z greater than zero
        phi_z_gt0 = np.arctan(point_on_R3_ellipse[:, 0] / point_on_R3_ellipse[:, 2])  # arctan(x / z)
        phi_z_lt0 = -np.pi - np.arctan(
            -point_on_R3_ellipse[:, 0] / point_on_R3_ellipse[:, 2]
        )  

        phi = np.where(z_gt0, phi_z_gt0, phi_z_lt0)
        
        # Undo phi rotation
        rotate_onto_yz_plane = np.empty((point_on_R3_ellipse.shape[0], 3))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
        for i in range(point_on_R3_ellipse.shape[0]):
            rotate_onto_yz_plane[i] = roty(-phi[i]) @ point_on_R3_ellipse[i]

        # Calculate theta
        theta = np.arctan(rotate_onto_yz_plane[:, 1] / rotate_onto_yz_plane[:, 2])

        # filter out points that are not within the bounds
        theta_mask = (theta > theta_bounds[0]) & (theta < theta_bounds[1])
        phi_mask = (phi > phi_bounds[0]) & (phi < phi_bounds[1])
        
        mask = theta_mask & phi_mask  
        
        final_points = np.vstack((final_points, point_on_R3_ellipse[mask]))
        angles = np.array([theta[mask],phi[mask]]).T
        final_angles = np.vstack((final_angles, angles))

        if final_points.shape[0] >= n_samples:
            final_points = final_points[:n_samples]            
            break

      
    return final_points, final_angles


In [29]:
N_SAMPLES = 2000

THETA_RANGE_ELLIPSOID = [-360,360]
PHI_RANGE_ELLIPSOID = [-360,360]
ELLIPSE_PARAMS = [10.5, 9, 6.35] #x, y, z

In [30]:
points, ellipsoid_angles = sample_points_ellipsoid(THETA_RANGE_ELLIPSOID, PHI_RANGE_ELLIPSOID,  N_SAMPLES, ELLIPSE_PARAMS)

plot_points(points,points)
